<a href="https://colab.research.google.com/github/compphoto/Intrinsic/blob/main/recoloring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/compphoto/Intrinsic
!cd Intrinsic/ && pip install .

In [ ]:
import torch

# import some helper functions from chrislib (will be installed by the intrinsic repo)
from chrislib.general import show, view, uninvert, match_scale
from chrislib.data_util import load_image

# import model loading and running the pipeline
from intrinsic.pipeline import run_gray_pipeline, load_models

In [ ]:
intrinsic_model = load_models('paper_weights')

In [ ]:
# three different example scenes from the paper
scene_name = 'yellow_chair'
# scene_name = 'brown_chairs'
# scene_name = 'spain_museum'

In [ ]:
inp = load_image(f'Intrinsic/examples/{scene_name}/input.png')[:, :, :3]
msk = load_image(f'Intrinsic/examples/{scene_name}/mask.png')[:, :, :3]
tex = load_image(f'Intrinsic/examples/{scene_name}/texture.png')[:, :, :3] ** 2.2

In [ ]:
results = run_gray_pipeline(
    intrinsic_model,
    inp,
    resize_conf=None,
    maintain_size=True
)

alb = results['albedo']
image = results['image']
inv_shd = results['inv_shading']

shd = uninvert(inv_shd)[:, :, None]

In [ ]:
def perform_recolor(msk, alb, shd, shd_power=1.0, recolor=None):
    # this function will perform the illumination-aware recoloring, or apply a shading curve
    # msk - numpy array (HxWx1) denoting the region to perform the edit
    # alb - linear albedo of the image
    # shd - linear shading of the image
    # shd_power - exponent to apply to the shading (<1 for more diffuse, >1 for more specular)
    # recolor - a texture to apply to the edited region, no recoloring is performed if set to None

    if recolor is None:
        our_new_alb = alb
    else:
        # we match the scale of the texture to the albedo in the edited region to
        # ensure the appearance of the region is maintained, but this can be altered
        recolor = match_scale(recolor, alb, msk.astype(bool))
        our_new_alb = ((1.0 - msk) * alb) + (msk * recolor)

    # apply exponentiation to the shading of the region and composite
    masked_shd = msk * (shd ** shd_power)
    new_shd = ((1.0 - msk) * shd) + masked_shd

    # combine edited albedo and shading, gamma correct and clip
    recolored = (our_new_alb * new_shd) ** (1/2.2)

    return recolored.clip(0, 1)

In [ ]:
# NOTE: setting the shading exponent to >1 will make the shading appear more specular,
# but small errors in the shading (albedo leakage) will be amplified in some cases
show(perform_recolor(msk, alb, shd, 1.0, recolor=tex))